In [14]:
import sys
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate
import datetime
import matplotlib.pyplot as plt

In [15]:
q='''select t1.ecosystem as ecosystem, advisories, advisory_with_CVEs, packages, fixing_releases
from
(select ecosystem, count(distinct a.id) as advisories, count(distinct p.id) as packages, count(distinct advisory_id, version) as fixing_releases
from advisory a
join package p on a.package_id = p.id
join fixing_releases fr on a.id = fr.advisory_id
where type != 'Malicious Package'
and version != 'manual checkup needed'
and ecosystem != 'cocoapods'
group by ecosystem) as t1
join
(select ecosystem, count(distinct a.id) as advisory_with_CVEs
from advisoryCVE aC
join advisory a on aC.advisory_id = a.id
join fixing_releases fr on a.id = fr.advisory_id
join package p on a.package_id = p.id
where type != 'Malicious Package'
and version != 'manual checkup needed'
and ecosystem != 'cocoapods'
group by ecosystem) as t2
on t1.ecosystem = t2.ecosystem
order by advisories desc;'''
df = pd.DataFrame(sql.execute(q))

In [18]:
df

,ecosystem,advisories,advisory_with_CVEs,packages,fixing_releases
0,Maven,1663,1406,916,2300
1,Composer,857,601,230,1735
2,npm,792,474,540,949
3,pip,624,450,313,906
4,NuGet,378,315,120,491
5,Go,236,191,184,326
6,RubyGems,226,173,124,376


In [19]:
df.loc[len(df)]=['Total',df['advisories'].sum(),df['advisory_with_CVEs'].sum(),df['packages'].sum(), df['fixing_releases'].sum()] 
df

,ecosystem,advisories,advisory_with_CVEs,packages,fixing_releases
0,Maven,1663,1406,916,2300
1,Composer,857,601,230,1735
2,npm,792,474,540,949
3,pip,624,450,313,906
4,NuGet,378,315,120,491
5,Go,236,191,184,326
6,RubyGems,226,173,124,376
7,Total,4776,3610,2427,7083


In [20]:
df['non-CVEs (%)'] = df['advisories'] - df['advisory_with_CVEs']
df['non-CVEs (%)'] = round (df['non-CVEs (%)'] / df['advisories'] * 100 , 2)
df

,ecosystem,advisories,advisory_with_CVEs,packages,fixing_releases,non-CVEs (%)
0,Maven,1663,1406,916,2300,15.45
1,Composer,857,601,230,1735,29.87
2,npm,792,474,540,949,40.15
3,pip,624,450,313,906,27.88
4,NuGet,378,315,120,491,16.67
5,Go,236,191,184,326,19.07
6,RubyGems,226,173,124,376,23.45
7,Total,4776,3610,2427,7083,24.41


In [21]:
print(tabulate(df, tablefmt='latex', headers='keys',showindex=False))

\begin{tabular}{lrrrrr}
\hline
 ecosystem   &   advisories &   advisory\_with\_CVEs &   packages &   fixing\_releases &   non-CVEs (\%) \\
\hline
 Maven       &         1663 &                 1406 &        916 &              2300 &          15.45 \\
 Composer    &          857 &                  601 &        230 &              1735 &          29.87 \\
 npm         &          792 &                  474 &        540 &               949 &          40.15 \\
 pip         &          624 &                  450 &        313 &               906 &          27.88 \\
 NuGet       &          378 &                  315 &        120 &               491 &          16.67 \\
 Go          &          236 &                  191 &        184 &               326 &          19.07 \\
 RubyGems    &          226 &                  173 &        124 &               376 &          23.45 \\
 Total       &         4776 &                 3610 &       2427 &              7083 &          24.41 \\
\hline
\end{tabular}
